In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm
import os, ast, json
from sklearn.model_selection import train_test_split

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
train_df, test_df = train_test_split(df, test_size=0.1)

train_df["annotations"] = train_df["annotations"].map(lambda x: ast.literal_eval(x))
test_df["annotations"] = test_df["annotations"].map(lambda x: ast.literal_eval(x))
train_df.head()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
def to_coco(df):
    coco_data = {
        "info":"kaaggle competition https://www.kaggle.com/c/tensorflow-great-barrier-reef",
        "licenses" : [{"id" : 1, "name":"kaggle", "url":"https://www.kaggle.com/c/tensorflow-great-barrier-reef"}],
        "categories" : [{"id" : 1, "name":"starfish"}],
        "images" : [],
        "annotations" : []
    }
    a_id = 0
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        image_path = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{}/{}.jpg".format(row["video_id"], row["video_frame"])
        coco_data["images"].append({
            "id" : idx,
            "width" : 1280,
            "height" : 720,
            "file_name" : image_path,
            "license" : 1
        })
        annotations = row["annotations"]
        for i, annotation in enumerate(annotations):
            coco_data["annotations"].append({
                "id" : a_id,#"{}_{}_{}".format(row["video_id"], row["video_frame"], i),
                "image_id" : idx,
                "area" : annotation["width"] * annotation["height"],
                "category_id" : 1,
                "iscrowd" : 1,
                "bbox" : [annotation["x"], annotation["y"], annotation["width"], annotation["height"]]
            })
            a_id += 1
    return coco_data

In [ ]:
train_coco = to_coco(train_df)
test_coco = to_coco(test_df)
#     break
# print(coco_data)
# train_df.shape, test_df.shape

In [ ]:
json.dump(train_coco, open("annotations_train.json", "w"))
json.dump(test_coco, open("annotations_val.json", "w"))

In [ ]:
# data = json.load(open("annotations_coco.json", "r"))

In [ ]:
# imgs ={}
# for img in data['images']:
#     imgs[img['id']] = img
# print(len(imgs.keys()))

In [ ]:
!pip install Cython termcolor numpy tensorboard pycocotools matplotlib pyaml opencv-python tqdm pytorch-lightning torchmetrics

In [ ]:
%%bash
git clone https://github.com/RangiLyu/nanodet.git
cd nanodet
python setup.py develop

In [ ]:
config = '# nanodet-plus-m-1.5x_416 \n \
# COCO mAP(0.5:0.95) = 0.341 \n \
#             AP_50  = 0.506 \n \
#             AP_75  = 0.357 \n \
#           AP_small = 0.143 \n \
#               AP_m = 0.363 \n \
#               AP_l = 0.539 \n \
save_dir: nanodet-plus-m-1.5x_416 \n \
model: \n \
  weight_averager: \n \
    name: ExpMovingAverager \n \
    decay: 0.9998 \n \
  arch: \n \
    name: NanoDetPlus \n \
    detach_epoch: 10 \n \
    backbone: \n \
      name: ShuffleNetV2 \n \
      model_size: 1.5x \n \
      out_stages: [2,3,4] \n \
      activation: LeakyReLU \n \
    fpn: \n \
      name: GhostPAN \n \
      in_channels: [176, 352, 704] \n \
      out_channels: 128 \n \
      kernel_size: 5 \n \
      num_extra_level: 1 \n \
      use_depthwise: True \n \
      activation: LeakyReLU \n \
    head: \n \
      name: NanoDetPlusHead \n \
      num_classes: 1 \n \
      input_channel: 128 \n \
      feat_channels: 128 \n \
      stacked_convs: 2 \n \
      kernel_size: 5 \n \
      strides: [8, 16, 32, 64] \n \
      activation: LeakyReLU \n \
      reg_max: 7 \n \
      norm_cfg: \n \
        type: BN \n \
      loss: \n \
        loss_qfl: \n \
          name: QualityFocalLoss \n \
          use_sigmoid: True \n \
          beta: 2.0 \n \
          loss_weight: 1.0 \n \
        loss_dfl: \n \
          name: DistributionFocalLoss \n \
          loss_weight: 0.25 \n \
        loss_bbox: \n \
          name: GIoULoss \n \
          loss_weight: 2.0 \n \
    # Auxiliary head, only use in training time. \n \
    aux_head: \n \
      name: SimpleConvHead \n \
      num_classes: 1 \n \
      input_channel: 256 \n \
      feat_channels: 256 \n \
      stacked_convs: 4 \n \
      strides: [8, 16, 32, 64] \n \
      activation: LeakyReLU \n \
      reg_max: 7 \n \
data: \n \
  train: \n \
    name: CocoDataset \n \
    img_path: "" \n \
    ann_path: /kaggle/working/annotations_train.json \n \
    input_size: [416,416] #[w,h] \n \
    keep_ratio: False \n \
    pipeline: \n \
      perspective: 0.0 \n \
      scale: [0.6, 1.4] \n \
      stretch: [[0.8, 1.2], [0.8, 1.2]] \n \
      rotation: 0 \n \
      shear: 0 \n \
      translate: 0.2 \n \
      flip: 0.5 \n \
      brightness: 0.2 \n \
      contrast: [0.6, 1.4] \n \
      saturation: [0.5, 1.2] \n \
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]] \n \
  val: \n \
    name: CocoDataset \n \
    img_path: "" \n \
    ann_path: /kaggle/working/annotations_val.json \n \
    input_size: [416,416] #[w,h] \n \
    keep_ratio: False \n \
    pipeline: \n \
      normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]] \n \
device: \n \
  gpu_ids: [0] \n \
  workers_per_gpu: 2 \n \
  batchsize_per_gpu: 4 \n \
schedule: \n \
#  resume: \n \
#  load_model: \n \
  optimizer: \n \
    name: AdamW \n \
    lr: 0.001 \n \
    weight_decay: 0.05 \n \
  warmup: \n \
    name: linear \n \
    steps: 500 \n \
    ratio: 0.0001 \n \
  total_epochs: 300 \n \
  lr_schedule: \n \
    name: CosineAnnealingLR \n \
    T_max: 300 \n \
    eta_min: 0.00005 \n \
  val_intervals: 10 \n \
grad_clip: 35 \n \
evaluator: \n \
  name: CocoDetectionEvaluator \n \
  save_key: mAP \n \
log: \n \
  interval: 50 \n \
 \n \
class_names: ["starfish"] \n '

with open("custom_config.yml", "w") as f:
    f.write(config)

In [ ]:
# %%bash
# # !cat custom_config.yml
# cd nanodet
!python nanodet/tools/train.py custom_config.yml